# Email Spam Filtering Project (Training) by using Naive Bayes Classifier

In [1]:
import pandas as pd
import numpy as np

In [2]:
VOCAB_SIZE = 2500
TRAINING_DATA_FILE = 'SpamData/02_Training/train_data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test_data.txt'

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter = ' ', dtype = int)
sparse_test_data =  np.loadtxt(TEST_DATA_FILE, delimiter = ' ', dtype = int)

In [4]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [5]:
sparse_test_data[:5]

array([[     8,      2,      1,  31206,      1],
       [     8,      3,      1, 124771,      4],
       [     8,      4,      1,  62224,      2],
       [     8,      5,      1,  31219,      1],
       [     8,      6,      1,  62171,      2]])

In [6]:
sparse_train_data.shape

(258802, 4)

In [7]:
sparse_test_data.shape

(117385, 5)

In [8]:
np.unique(sparse_train_data[:,0]).size

4013

In [9]:
np.unique(sparse_test_data[:,0]).size

1725

In [10]:
# at selects a single scalar value in the DataFrame by label only.
# .iat selects a single scalar value in the DataFrame by integerlocation only.

In [11]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx = 0, word_idx = 1, cat_idx = 2, freq_idx = 3):
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    
    full_matrix = pd.DataFrame(index = doc_id_names, columns = column_names)
    full_matrix.fillna(value = 0, inplace = True)
    
    
    for i in range(sparse_matrix.shape[0]):
        
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurence
        
    full_matrix.set_index('DOC_ID', inplace = True)
    
    return full_matrix

In [12]:
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)
full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,1,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5792,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5793,0,2,2,0,1,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,9,1,0,0,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
full_train_data.CATEGORY.size

4013

In [14]:
full_train_data.CATEGORY.sum()

1246

In [16]:
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
prob_spam

0.3104909045601794

In [17]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5792,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5793,2,2,0,1,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,1,9,1,0,0,1,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
email_length = full_train_features.sum(axis = 1)
email_length

DOC_ID
0        87
1        53
2        40
3       183
4        43
       ... 
5792     46
5793     30
5794    228
5795     57
5796    264
Length: 4013, dtype: int64

In [19]:
total_wc = email_length.sum()
total_wc

429218

In [20]:
spam_length = email_length[full_train_data.CATEGORY == 1]
spam_length

DOC_ID
0        87
1        53
2        40
3       183
4        43
       ... 
1885     45
1887     26
1889    115
1890     26
1895    162
Length: 1246, dtype: int64

In [22]:
spam_wc = spam_length.sum()
spam_wc

176299

In [23]:
ham_length = email_length[full_train_data.CATEGORY == 0]
ham_length

DOC_ID
1896     82
1898     41
1899     41
1900    112
1901     66
       ... 
5792     46
5793     30
5794    228
5795     57
5796    264
Length: 2767, dtype: int64

In [24]:
nonspam_wc = ham_length.sum()
nonspam_wc

252919

In [25]:
spam_wc + nonspam_wc

429218

In [26]:
# average length of spam mail
spam_wc/len(spam_length)

141.491974317817

In [27]:
# average length of ham mail
nonspam_wc/len(ham_length)

91.40549331405855

In [29]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
summed_spam_tokens = train_spam_tokens.sum(axis = 0) + 1
summed_spam_tokens

0       2177
1        935
2       1217
3       2020
4       1214
        ... 
2495       7
2496      15
2497      10
2498      24
2499       6
Length: 2500, dtype: int64

In [32]:
prob_spam_tokens = summed_spam_tokens/(spam_wc + VOCAB_SIZE)
prob_spam_tokens

0       0.012176
1       0.005229
2       0.006807
3       0.011298
4       0.006790
          ...   
2495    0.000039
2496    0.000084
2497    0.000056
2498    0.000134
2499    0.000034
Length: 2500, dtype: float64

In [33]:
prob_spam_tokens.sum()

1.0

In [34]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1900,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1901,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5792,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5793,2,2,0,1,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,1,9,1,0,0,1,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [36]:
summed_ham_tokens = train_ham_tokens.sum(axis = 1) + 1
summed_ham_tokens 

DOC_ID
1896     83
1898     42
1899     42
1900    113
1901     67
       ... 
5792     47
5793     31
5794    229
5795     58
5796    265
Length: 2767, dtype: int64

In [38]:
prob_token_nonspam = summed_ham_tokens/(nonspam_wc + VOCAB_SIZE)
prob_token_nonspam

DOC_ID
1896    0.000325
1898    0.000164
1899    0.000164
1900    0.000442
1901    0.000262
          ...   
5792    0.000184
5793    0.000121
5794    0.000897
5795    0.000227
5796    0.001038
Length: 2767, dtype: float64

In [39]:
prob_token_nonspam.sum()

1.001045341184485

In [40]:
prob_token_all = full_train_features.sum(axis = 1)/total_wc
prob_token_all

DOC_ID
0       0.000203
1       0.000123
2       0.000093
3       0.000426
4       0.000100
          ...   
5792    0.000107
5793    0.000070
5794    0.000531
5795    0.000133
5796    0.000615
Length: 4013, dtype: float64

In [41]:
prob_token_all.sum()

1.0

In [42]:
TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam-txt'
TOKEN_NONSPAM_PROB_FILE = 'SpamData/03_Testing/prob-ham-txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens-txt'

np.savetxt(TOKEN_SPAM_PROB_FILE, prob_spam_tokens, fmt='%.18e')
np.savetxt(TOKEN_NONSPAM_PROB_FILE, prob_token_nonspam, fmt='%.18e')
np.savetxt(TOKEN_ALL_PROB_FILE, prob_token_all, fmt='%.18e')

In [43]:
sparse_test_data[:5]

array([[     8,      2,      1,  31206,      1],
       [     8,      3,      1, 124771,      4],
       [     8,      4,      1,  62224,      2],
       [     8,      5,      1,  31219,      1],
       [     8,      6,      1,  62171,      2]])

In [44]:
full_test_data = make_full_matrix(sparse_test_data, nr_words = VOCAB_SIZE)
full_test_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,31206,124771,62224,31219,62171,124609,31086,...,0,0,0,0,0,0,0,0,0,0
12,1,674274,112397,112395,0,112368,0,0,112372,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,15071,15065,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,55383,27654,27722,55556,0,0,83354,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,97305,0,97313,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5781,0,0,0,0,0,0,790467,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5784,0,221320,110761,0,221662,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5789,0,0,320867,0,160237,320769,240369,240078,80012,320933,...,0,0,0,0,0,0,0,0,0,0


In [48]:
x_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
x_test

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,0,0,31206,124771,62224,31219,62171,124609,31086,62192,...,0,0,0,0,0,0,0,0,0,0
12,674274,112397,112395,0,112368,0,0,112372,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,15071,15065,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,55383,27654,27722,55556,0,0,83354,0,110807,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,97305,0,97313,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5781,0,0,0,0,0,790467,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5784,221320,110761,0,221662,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5789,0,320867,0,160237,320769,240369,240078,80012,320933,240960,...,0,0,0,0,0,0,0,0,0,0


In [50]:
y_test = full_test_data.CATEGORY
y_test

DOC_ID
8       1
12      1
14      1
15      1
17      1
       ..
5781    0
5784    0
5789    0
5790    0
5791    0
Name: CATEGORY, Length: 1725, dtype: int64

In [51]:
TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

In [52]:
np.savetxt(TEST_FEATURE_MATRIX, x_test)
np.savetxt(TEST_TARGET_FILE, y_test)